In [1]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'satellite-methane',
    {
        'processing_level': 'level_2',
        'variable': 'ch4',
        'sensor_and_algorithm': 'iasi_metop_b_nlis',
        'year': '2019',
        'month': '07',
        'day': '11',
        'version': '9.1',
        'format': 'zip',
    },
    '/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/GitHub/Climate_Coders/data/download.zip')

2023-07-11 13:56:03,288 INFO Welcome to the CDS
2023-07-11 13:56:03,289 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/satellite-methane
2023-07-11 13:56:03,560 INFO Request is completed
2023-07-11 13:56:03,567 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data3/dataset-satellite-methane-6ce01e3c-4218-47a9-89af-29a9f31211b4.zip to /Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/GitHub/Climate_Coders/data/download.zip (313.8K)
2023-07-11 13:56:04,661 INFO Download rate 296.1K/s


Result(content_length=321320,content_type=application/zip,location=https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data3/dataset-satellite-methane-6ce01e3c-4218-47a9-89af-29a9f31211b4.zip)

# Test that the code itself runs

In [1]:
## Get location 

from geopy.geocoders import Nominatim

# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")

# entering the location name
getLoc = loc.geocode("my_location")

target_address = getLoc.address
target_lon = getLoc.longitude
#target_lon = round(target_lon, 2)
target_lat = getLoc.latitude
#target_lat = round(target_lat, 2)

# Find closest methane observation 

import netCDF4 as nc
import pandas as pd
from geopy.distance import geodesic

# Open the netCDF file
nc_file = nc.Dataset('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/GitHub/Climate_Coders/data/20190711-C3S-L2_GHG-GHG_PRODUCTS-IASI-METOPB-NLIS-DAILY-v9.1.nc')

# Get the required variables from the netCDF file
time = nc_file.variables['time'][:]
ch4 = nc_file.variables['ch4'][:]
lat = nc_file.variables['latitude'][:]
lon = nc_file.variables['longitude'][:]

# Create a DataFrame
df = pd.DataFrame({'latitude': lat, 'longitude': lon, 'ch4': ch4, 'time': time})
#df['time'] = pd.to_datetime(df['time'], origin='1970-01-01', unit='D') # ADDED


# Function to calculate distance between two points
def calculate_distance(lon1, lat1, lon2, lat2):
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    distance = geodesic(point1, point2).miles
    return distance

# Calculate distances between target point and all points in the data frame
distances = df.apply(lambda row: calculate_distance(target_lon, target_lat, row['longitude'], row['latitude']), axis=1)

# Find the index of the closest point
closest_index = distances.idxmin()

# Retrieve the closest matching point
closest_point = df.loc[closest_index]

# Print the result
print(closest_point) # target_address, 

latitude    -1.590000e+00
longitude    3.104000e+01
ch4          1.841470e+03
time         1.562873e+09
Name: 10797, dtype: float64


# Do dashboard 

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from geopy.geocoders import Nominatim
import netCDF4 as nc
import pandas as pd
from geopy.distance import geodesic

# Open the netCDF file
nc_file = nc.Dataset('/Users/carolinkroeger/Library/CloudStorage/OneDrive-Nexus365/GitHub/Climate_Coders/data/20190711-C3S-L2_GHG-GHG_PRODUCTS-IASI-METOPB-NLIS-DAILY-v9.1.nc')

# Get the required variables from the netCDF file
time = nc_file.variables['time'][:]
ch4 = nc_file.variables['ch4'][:]
lat = nc_file.variables['latitude'][:]
lon = nc_file.variables['longitude'][:]

# Create a DataFrame
df = pd.DataFrame({'latitude': lat, 'longitude': lon, 'ch4': ch4, 'time': time})
#df['time'] = pd.to_datetime(df['time'], origin='1970-01-01', unit='D')


# Create a Dash app
app = dash.Dash(__name__)

# Dash layout
app.layout = html.Div([
    html.H1('Geolocation Dashboard'),
    html.Label('Enter a location:'),
    dcc.Input(id='location-input', type='text', placeholder='Enter a location'),
    html.Div(id='result-output')
])

# Dash callback
@app.callback(
    dash.dependencies.Output('result-output', 'children'),
    [dash.dependencies.Input('location-input', 'value')]
)
def update_output(value):
    if value:
        # calling the Nominatim tool
        loc = Nominatim(user_agent="GetLoc")
        
        # entering the location name
        getLoc = loc.geocode(value, timeout=5)

        target_address = getLoc.address
        target_lon = getLoc.longitude
        target_lat = getLoc.latitude

        # Function to calculate distance between two points
        def calculate_distance(lon1, lat1, lon2, lat2):
            point1 = (lat1, lon1)
            point2 = (lat2, lon2)
            distance = geodesic(point1, point2).miles
            return distance

        # Calculate distances between target point and all points in the data frame
        distances = df.apply(lambda row: calculate_distance(target_lon, target_lat, row['longitude'], row['latitude']), axis=1)

        # Find the index of the closest point
        closest_index = distances.idxmin()

        # Retrieve the closest matching point
        closest_point = df.loc[closest_index]
        #closest_point['latitude'] = closest_point['latitude'].round(2)
        #closest_point['longitude'] = closest_point['longitude'].round(2)
        #closest_point['ch4'] = closest_point['ch4'].round(2)

        # Return the result
        return html.Div([
            html.H3('Target Address: {}'.format(target_address)),
            html.H4('Closest Point: {}'.format(closest_point))
        ])

    return ''

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

/var/folders/fw/sfmxzmg52fg2gpxsf9nk9x300000gn/T/ipykernel_11568/2802583450.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/fw/sfmxzmg52fg2gpxsf9nk9x300000gn/T/ipykernel_11568/2802583450.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
